<h1><center>Ray Tracing Project Pt. 1</center></h1>

Some basic imports:

In [ ]:
from tqdm.notebook import tqdm
import os
import time
import numpy as np
import matplotlib.pyplot as plt

The following function generates the world of choice given a key value. 
Worlds are organized in order of complexity.
Objects and textures are referenced here without explanation, check the .py files to see how each must be called if you are interested in generating your own object.

In [ ]:
#Import files from src folder
from src.objects import *
from src.textures import *
from src.bvh import *
from src.materials import *



def world_generator(key,speedup=True):
    """
    world_generator generates a 'world' for the program to render.

    :param key: This is an integer that acts as the key to a specific world
    :optional param speedup: This is a boolean value that determines whether 
                             the boundary box method should be used to speedup 
                             the search.
    :return: This function returns either a bvh node or a hittable_list, both of
             of which act as something that the ray_color function can send rays
             to. It also returns the length of the world, which is only used when
             plotting performance to track how the bvh speedup performs as a 
             function of the total number of objects in the world.
    """ 
    world = hittable_list()
    if key == 1:
        #this world is a simple diffuse sphere
        world.add_object(sphere(vec3(0,0,-1),0.5,lambertian(solid_color(vec3(0.5,0.5,0.5)))))
        world.add_object(sphere(vec3(0,0,-1),0.5,lambertian(solid_color(vec3(0.5,0.5,0.5)))))
        world.add_object(sphere(vec3(0,-100.5,-1),100,lambertian(solid_color(vec3(0.5,0.5,0.5)))))
        t0 = 0
        tf = 1
        
    elif key == 2:
        #this world is three spheres, one diffuse, one fuzzy metal and one less fuzzy metal
        t0 = 0
        tf = 1
        material_ground = lambertian(solid_color(vec3(0.8, 0.8, 0.0)))
        material_center = lambertian(solid_color(vec3(0.7, 0.3, 0.3)))
        material_left   = metal(vec3(0.8, 0.8, 0.8), 0.0)
        material_right  = metal(vec3(0.8, 0.6, 0.2), 1.0)
        world.add_object(sphere(vec3( 0.0, -100.5, -1.0), 100.0, material_ground))
        world.add_object(sphere(vec3( 0.0,0.0, -1.0),   0.5, material_center))
        world.add_object(sphere(vec3(-1.0,0.0,-1.0),   0.5, material_left))
        world.add_object(sphere(vec3(1.0,0.0,-1.0),   0.5, material_right))
        
    elif key == 3:
        #make the sphere on the left glass, change the fuzziness of the sphere on the right
        #this world is three spheres, one diffuse, one fuzzy metal and one less fuzzy metal
        t0 = 0
        tf = 1
        material_ground = lambertian(solid_color(vec3(0.8, 0.8, 0.0)))
        material_center = lambertian(solid_color(vec3(0.7, 0.3, 0.3)))
        material_left   = dielectric(1.5)
        material_right  = metal(vec3(0.8, 0.6, 0.2), 0)
        world.add_object(sphere(vec3( 0.0, -100.5, -1.0), 100.0, material_ground))
        world.add_object(sphere(vec3( 0.0,0.0, -1.0),   0.5, material_center))
        world.add_object(sphere(vec3(-1.0,0.0,-1.0),   0.5, material_left))
        world.add_object(sphere(vec3(1.0,0.0,-1.0),   0.5, material_right))
    elif key == 4:
        #a bunch of spheres as a demonstration
        ground_material = lambertian(solid_color(vec3(0.9,0.9,0.9)))
        world.add_object(sphere(vec3(0,-1000,0),1000,ground_material))
        
        for a in range(-11,11,1):
            for b in range(-11,11,1):
                choose_mat = random.random()
                center = vec3(a+0.9*random.random(),0.2,b+0.9*random.random())
                if np.linalg.norm(center - vec3(4,0.2,0)) > 0.9:
                    if choose_mat < 0.6:
                        #diffuse
                        albedo = np.random.uniform(0,1,3)*np.random.uniform(0,1,3)
                        sphere_material = lambertian(solid_color(albedo))
                        world.add_object(sphere(center,0.2,sphere_material))
                    elif choose_mat <0.85:
                        #metal
                        albedo = np.random.uniform(0.5,1,3)
                        fuzz = np.random.uniform(0,0.5)
                        sphere_material = metal(albedo,fuzz)
                        world.add_object(sphere(center,0.2,sphere_material))
                    else:
                        #glass
                        sphere_material = dielectric(1.5)
                        world.add_object(sphere(center,0.2,sphere_material))
                        
                        
        world.add_object(sphere(vec3(0,1,0),1.0,dielectric(1.5)))
        world.add_object(sphere(vec3(-4,1,0),1.0,lambertian(solid_color(vec3(0.4,0.2,0.1)))))
        world.add_object(sphere(vec3(4,1,0),1.,metal(vec3(0.7,0.6,0.5),0.0)))
        t0 = 0
        tf = 1
        
      
    #now play around with textures
    elif key == 5:
        #two checkered spheres
        checker = checker_texture((vec3(0.50196078431,0,0)),(vec3(0,0,0.8)))
        world.add_object(sphere(vec3(0,-10,0),10,lambertian(checker)))
        world.add_object(sphere(vec3(0,10,0),10,lambertian(checker)))
        t0 = 0
        tf = 1
        
    elif key == 6:
        #perlin noise (random generative) can be used as a texture
        pertext = noise_texture(4)
        world.add_object(sphere(vec3(0,-1000,0), 1000,lambertian(pertext)))
        world.add_object(sphere(vec3(0, 2, 0), 2,lambertian(pertext)))
        t0 = 0
        tf = 1
    
    elif key == 7:
        #can make images as textures as well
        image = image_texture('./img/pattern.jpg')
        world.add_object(sphere(vec3(0,0,-1),0.5,lambertian(image)))
        world.add_object(sphere(vec3(0,-100.5,-1),100,lambertian(solid_color(vec3(0.5,0.5,0.5)))))
        t0 = 0
        tf = 1
        
    elif key == 8:
        #experiment with lighting inside the world. rectangles also introduced here
        pertext = noise_texture(10)
        world.add_object(sphere(vec3(0,-1000,0),1000,lambertian(image_texture('./img/pattern.jpg'))))
        world.add_object(sphere(vec3(0,2,0),2,lambertian(pertext)))
        t0 = 0
        tf = 1
        difflight = diffuse_light(solid_color(vec3(10,10,10)))
        rect = xy_rect(3,5,1,3,-2,difflight)
        world.add_object(rect)
        sphere_light = sphere(vec3(1,8,0),2,difflight)
        world.add_object(sphere_light)

        
    elif key == 9:
        #play around with images, interior lighting and perlin noise texture
        t0 = 0
        tf = 1
        pertext = noise_texture(1)
        sun = diffuse_light(image_texture('./img/sun.jpg'))
        jupiter = image_texture('./img/jupiter.jpg')
        earth = image_texture('./img/earthmap.jpg')
        world.add_object(sphere(vec3(0,0,-53),50,sun))
        world.add_object(sphere(vec3(-3,3,2),2,lambertian(pertext)))
        world.add_object(sphere(vec3(0,0,2),2,lambertian(jupiter)))
        world.add_object(sphere(vec3(1,-3,2),1,lambertian(earth)))
        
    #Now mess around with the famous Cornell Box
    elif key == 10:
        #the classic cornell box
        red = lambertian(solid_color(vec3(0.65,0.05,0.05)))
        white = lambertian(solid_color(vec3(0.73,0.73,0.73)))
        green = lambertian(solid_color(vec3(0.12,0.45,0.15)))
        light = diffuse_light(solid_color(vec3(15,15,15)))
        world.add_object(yz_rect(0,555,0,555,555,red))
        world.add_object(yz_rect(0,555,0,555,0,green))
        world.add_object(xz_rect(213,343,227,332,554,light))
        world.add_object(xz_rect(0,555,0,555,0,white))
        world.add_object(xz_rect(0,555,0,555,555,white))
        world.add_object(xy_rect(0,555,0,555,555,white))
        t0 = 0
        tf = 1
        box1 = box(vec3(0,0,0),vec3(165,330,165),white)
        box1 = rotate_y(box1,15)
        box1 = translate(box1,vec3(265,0,295))
        world.add_object(box1)
        
        box2 = box(vec3(0,0,0),vec3(165,165,165),white)
        box2 = rotate_y(box2,-18)
        box2 = translate(box2,vec3(130,0,65))
        world.add_object(box2)
        
    elif key == 11:
        #a cornell box with one box made out of metal and a larger light, to test reflections
        t0 = 0
        tf = 1
        red = lambertian(solid_color(vec3(0.65,0.05,0.05)))
        white = lambertian(solid_color(vec3(0.73,0.73,0.73)))
        green = lambertian(solid_color(vec3(0.12,0.45,0.15)))
        light = diffuse_light(solid_color(vec3(15,15,15)))
        
        world.add_object(yz_rect(0,555,0,555,555,green))
        world.add_object(yz_rect(0,555,0,555,0,red))
        
        world.add_object(flip_face(xz_rect(213,343,227,332,554,light)))
        world.add_object(xz_rect(0,555,0,555,555,white))
        world.add_object(xz_rect(0,555,0,555,0,white))
        world.add_object(xy_rect(0,555,0,555,555,white))
        #new_world.add_object(xz_rect(213, 343, 227, 332, 554, light));
        
        aluminum = metal(vec3(0.8,0.8,0.8),0.0)
        box1 = box(vec3(0,0,0),vec3(165,330,165),aluminum)
        box1 = rotate_y(box1,25)
        box1 = translate(box1,vec3(265,0,295))

        world.add_object(box1)
        box2 = box(vec3(0,0,0),vec3(165,165,165),white)
        box2 = rotate_y(box2,-20)
        box2 = translate(box2,vec3(130,0,65))
        world.add_object(box2)
        
    elif key == 12:
        #cornell box with a larger light,a box made out of smoke, and a sphere made out of smoke 
        t0 = 0
        tf = 1
        red = lambertian(solid_color(vec3(0.65,0.05,0.05)))
        white = lambertian(solid_color(vec3(0.73,0.73,0.73)))
        green = lambertian(solid_color(vec3(0.12,0.45,0.15)))
        light = diffuse_light(solid_color(vec3(15,15,15)))
        
        world.add_object(yz_rect(0,555,0,555,555,green))
        world.add_object(yz_rect(0,555,0,555,0,red))
        world.add_object(xz_rect(113,443,127,432,554,light))
        world.add_object(xz_rect(0,555,0,555,555,white))
        world.add_object(xz_rect(0,555,0,555,0,white))
        world.add_object(xy_rect(0,555,0,555,555,white))
        
        
        box1 = box(vec3(0,0,0),vec3(165,330,165),white)
        box1 = rotate_y(box1,15)
        box1 = translate(box1,vec3(265,0,295))
        smokebox1 = constant_medium(box1,0.01,solid_color(vec3(0,0,0)))
        world.add_object(smokebox1)
        
        sphere1 = sphere(vec3(190,90,190),90,white)
        smokeball = constant_medium(sphere1,0.01,solid_color(vec3(1,1,1)))
        
        world.add_object(smokeball)

    elif key == 13:
        #this is the cornell box with a glass sphere and an aluminum box. 
        #used for comparing noise to the pdf_method in the second notebook
        t0 = 0
        tf = 1
        red = lambertian(solid_color(vec3(0.65,0.05,0.05)))
        white = lambertian(solid_color(vec3(0.73,0.73,0.73)))
        green = lambertian(solid_color(vec3(0.12,0.45,0.15)))
        light = diffuse_light(solid_color(vec3(15,15,15)))
        
        world.add_object(yz_rect(0,555,0,555,555,green))
        world.add_object(yz_rect(0,555,0,555,0,red))
        world.add_object(flip_face(xz_rect(213,343,227,332,554,light)))
        world.add_object(xz_rect(0,555,0,555,555,white))
        world.add_object(xz_rect(0,555,0,555,0,white))
        world.add_object(xy_rect(0,555,0,555,555,white))
        
        
        aluminum = metal(vec3(0.8,0.8,0.8),0.0)
        box1 = box(vec3(0,0,0),vec3(165,330,165),aluminum)
        box1 = rotate_y(box1,25)
        box1 = translate(box1,vec3(265,0,295))
        world.add_object(box1)
        
        glass = dielectric(1.5)
        world.add_object(sphere(vec3(190,90,190),90,glass))
        world.add_object(sphere(vec3(300,20,100),20,aluminum))

    else:
        raise ValueError("Key must be an integer in range of 1-13")
        
    length = len(world.data)
    if speedup:
        bvh = bvh_node(world.data,0,length,t0,tf)
        final = bvh     
    else:
        final = world
    return final,length


In [ ]:
from src.camera import *
from src.ray import *
from src.image_writing import *

def main(key,aspectRatio,image_width,samps_per_pix,depth,speedup=True,write=True):
    """
    main: This is the driver function that actually performs the render

    :param key: This is an integer that acts as the key to a specific world.
    :param aspectRatio: The desired aspect ratio of the image
    :param image_width: The desired width of image to test on.
    :param samps_per_pix: This is the number of rays to send to each pixel
    :param depth: This is how far to follow each ray before we assume it disappears
    
    :optional: speedup: This defines whether the boundary box method is used at 
                        construction. It is a boolean value.
    :optional: write: This defines whether or not you want to write the image to
                      the image director. It is a boolean value. The only time
                      you'd really want this false is if you're measuring performance
                      and don't care about image output.
                        
    :return: The function returns the length of the world, mostly so the 
             measure_performance function can put that information on the plot.
    """    
    
    #Set up Image
    #default parameters, can change depending on key
    aspect_ratio = aspectRatio
    width = image_width #1200 #for full size image
    height = int(width/aspect_ratio)
    samples_per_pixel = samps_per_pix #increasing this will increase runtime
    max_depth = depth #how many times we'll follow a single ray before it returns no light
    
    
    
    world_len = 0
    
    #Generate world
    world,world_len = world_generator(key,speedup)
    
    
    
    
    #Initialize Camera
    
    if key == 1:
        background = vec3(0.7,0.8,1.0) #sky like color
        lookfrom = vec3(0,0.2,1)
        lookat = vec3(0,0,0)
        vup = vec3(0,1,0)
        dist_to_focus = 10.
        aperture = 0
        vfov = 40
        
    elif key == 2:
        background = vec3(0.7,0.8,1.0) #sky like color
        lookfrom = vec3(0,0.2,2)
        lookat = vec3(0,0,0)
        vup = vec3(0,1,0)
        dist_to_focus = 10.
        aperture = 0
        vfov = 40
    
    elif key == 3:
        background = vec3(0.7,0.8,1.0) #sky like color
        lookfrom = vec3(0,0.2,2)
        lookat = vec3(0,0,0)
        vup = vec3(0,1,0)
        dist_to_focus = 10.
        aperture = 0
        vfov = 40
        
    elif key == 4:
        background = vec3(0.7,0.8,1.0)
        lookfrom = vec3(13,2,3)
        lookat = vec3(0,0,0)
        vup = vec3(0,1,0)
        dist_to_focus = 10.
        aperture = 0.1
        vfov = 20
    
    elif key == 5:
        background = vec3(0.7,0.8,1.0)
        lookfrom = vec3(13,2,3)
        lookat = vec3(0,0,0)
        vfov = 20
        vup = vec3(0,1,0)
        dist_to_focus = 10.
        aperture = 0
        
    elif key == 6:
        background = vec3(0.7,0.8,1.0)
        lookfrom = vec3(13,2,3)
        lookat = vec3(0,0,0)
        vfov = 28
        vup = vec3(0,1,0)
        dist_to_focus = 10.
        aperture = 0
    
    elif key == 7:
        background = vec3(0.7,0.8,1.0)
        lookfrom = vec3(0,0.2,1)
        lookat = vec3(0,0,0)
        vup = vec3(0,1,0)
        dist_to_focus = 10.
        aperture = 0
        vfov = 40
        
    elif key == 8:
        background = vec3(0,0,0) #black as the default for interior lighting
        lookfrom = vec3(26,3,6)
        lookat = vec3(0,2,0)
        vfov = 20
        vup = vec3(0,1,0)
        dist_to_focus = 10.
        aperture = 0
    
    elif key == 9:
        background = vec3(0,0,0)
        lookfrom = vec3(26,0,6)
        lookat = vec3(0,0,0)
        vfov = 20
        vup = vec3(0,1,0)
        dist_to_focus = 10.
        aperture = 0
    
    elif key == 10:
        background = vec3(0,0,0)
        lookfrom = vec3(278,278,-800)
        lookat = vec3(278,278,0)
        vup = vec3(0,1,0)
        vfov = 40
        dist_to_focus = 10
        aperture = 0
        
    elif key == 11:
        background = vec3(0,0,0)
        lookfrom = vec3(278,278,-800)
        lookat = vec3(278,278,0)
        vup = vec3(0,1,0)
        vfov = 40
        dist_to_focus = 10
        aperture = 0
    elif key == 12:
        background = vec3(0,0,0)
        lookfrom = vec3(278,278,-800)
        lookat = vec3(278,278,0)
        vup = vec3(0,1,0)
        vfov = 40
        dist_to_focus = 10
        aperture = 0
    elif key == 13:
        background = vec3(0,0,0)
        lookfrom = vec3(278,278,-800)
        lookat = vec3(278,278,0)
        vup = vec3(0,1,0)
        vfov = 40
        dist_to_focus = 10
        aperture = 0

        

        
        
        
    cam = camera(lookfrom,lookat,vup,vfov,aspect_ratio,aperture,dist_to_focus,0,1)
    
    
    
    #Render the image
    lines = []
    for  j in tqdm(range(height,0,-1)):
        for i in range(width):
            pixel_color = vec3(0.,0.,0.)
            for s in range(samples_per_pixel):
                u = (i + random.random())/(width-1)
                v = (j+random.random())/(height-1)
                r = cam.get_ray(u,v)
                pixel_color += ray_color(r,background,world,max_depth)
            line = write_color(pixel_color,samples_per_pixel)
            lines.append(line)
     
    if write:
        image = open(f"./renders/world_{key}.ppm", "w")
        header = "".join(["P3\n",str(width),' ',str(height),"\n255\n"])
        image.write(header)
        image.writelines(lines)
        image.close()

        #Note: image magick is a required to convert the image to a jpeg
        #installation is straightforward with macports or homebrew (brew install imagemagick)
        #if you do not have image magick, the image will output as a PPM file
        #set convert to 0 if you are okay with that and don't want to install image magick
        convert = 1
        if convert == 1:
            to_jpg = f'convert ./renders/world_{key}.ppm ./renders/world_{key}.jpg'
            os.system(to_jpg)
            os.remove(f'./renders/world_{key}.ppm')
    return world_len
    

Suggested Aspect Ratios depending on key
<p>keys 1-9: 3:2, 16:9</p>
<p>keys 10-12: 1:1</p>

In [ ]:
width = 400
aspectRatio = 1
key = 11
depth = 50
speedup=True
samples_per_pix=5
item_count = main(key,aspectRatio,width,samples_per_pix,depth,speedup)
print(f'Number of items in world: {item_count}')

The above code used simple monte carlo methods to render images. Now I want to try and reduce noise/how I sample. This involves importing new files, as these new sampling methods are not entirely compatible with everything that the above code could do. So, check this folder for ray_tracing_pdf.ipynb and continue from there!